In [6]:
import os
import numpy as np
import pandas as pd

In [7]:
def get_excel_files(directory):
    excel_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(('.xlsx', '.xls')):  # Mengecek file dengan ekstensi .xlsx atau .xls
                file_path = os.path.join(root, file)
                excel_files.append(file_path)
    return excel_files

In [8]:
def contains_total(data_list):
    # Iterate over each element in the list
    for item in data_list:
        # Check if "TOTAL" is in the current item (case-insensitive)
        if "TOTAL" in item.upper():
            return True
    return False
    
def read_excel_until_total(file_path, start_row=0):
    # Load the Excel file
    xls = pd.ExcelFile(file_path)

    # Initialize a list to store data from all sheets
    all_data = []

    # Iterate through each sheet in the Excel file
    for i, sheet_name in enumerate(xls.sheet_names):
        # Read the sheet into a DataFrame
        df = pd.read_excel(xls, sheet_name=sheet_name,header=0)

        # # Filter rows starting from `start_row`
        filtered_data = []

        for index, row in df.iterrows():
            filtered_data.append(row)

        # Convert the list of rows back into a DataFrame
        if filtered_data:  # Proceed only if there is data to add
            sheet_data = pd.DataFrame(filtered_data)
            sheet_data['SHEET NAME'] = sheet_name  # Add the sheet name to the DataFrame
            sheet_data['TANGGAL'] = sheet_name

            # Append the filtered data from this sheet to all_data
            all_data.append(sheet_data)

    
    # Concatenate all the data from all sheets
    result_df = pd.concat(all_data, ignore_index=True)     
    result_df.columns = result_df.columns.str.upper()
    result_df['JUMLAH'] = result_df['TOTAL']
    return result_df

def penyesuaian_format(df):
    
    df['WAKTU'] = df['WAKTU'].str.split(' - ', expand=True)[0].str.zfill(5)
    
    # Menambahkan tanggal, bulan, dan tahun ke waktu
    df['WAKTU'] = thn_bln + df['TANGGAL'] + ' ' + df['WAKTU']  # Tambahkan tanggal default
    
    # Mengubah kolom 'WAKTU' menjadi datetime dengan format lengkap
    df['WAKTU'] = pd.to_datetime(df['WAKTU'], format='%Y-%m-%d %H:%M')
    
    # Filter data berdasarkan apakah menit adalah kelipatan dari 60
    df = df[df['WAKTU'].dt.minute % 60 == 0]
    return df

def pisahkan_kolom_waktu(df):
    # Pastikan kolom WAKTU sudah dalam format datetime
    df['WAKTU'] = pd.to_datetime(df['WAKTU'])

    # Pisahkan kolom WAKTU menjadi beberapa kolom
    df['TAHUN'] = df['WAKTU'].dt.year
    df['BULAN'] = df['WAKTU'].dt.month
    df['TANGGAL'] = df['WAKTU'].dt.day
    df['HARI'] = df['WAKTU'].dt.day_name()
    df['JAM'] = df['WAKTU'].dt.strftime('%H:%M')
    # df = df.drop(columns=['INTERVAL','UPPKB','SHEET NAME'])
    
    # Daftar kolom yang diinginkan dalam urutan baru
    new_column_order = [
        'WAKTU', 'TAHUN', 'BULAN', 'TANGGAL', 'HARI', 'JAM', 
        'MOTOR', 'MOBIL', 'TRUK/BUS', 'JUMLAH'
    ]

    # Menyusun ulang kolom berdasarkan urutan yang diinginkan
    df = df.reindex(columns=new_column_order)
    return df

In [9]:
def mencari_data_hilang(start, end, df, nama_file):        
    # Membuat rentang tanggal untuk seluruh bulan
    dates_range = pd.date_range(start=start, end=end, freq='D')
    
    # Inisialisasi list untuk menyimpan DataFrame dari waktu yang hilang
    missing_times_list = []
    
    # # Loop melalui setiap hari dalam rentang bulan
    for date in dates_range:
        day = date.date()
        
        # Filter data untuk hari ini
        daily_data = df[df['WAKTU'].dt.date == day]
        
        # Buat rentang waktu yang lengkap untuk hari ini
        full_daily_range = pd.date_range(start=f'{day} 00:00', end=f'{day} 23:00', freq='60T')
        
        # Identifikasi waktu yang hilang
        missing_daily_times = full_daily_range[~full_daily_range.isin(daily_data['WAKTU'])]
        
        # Jika ada waktu yang hilang, tambahkan ke list missing_times_list
        if not missing_daily_times.empty:
            missing_times_df = pd.DataFrame({
                'WAKTU': missing_daily_times,
                'MOTOR': np.nan,
                'MOBIL': np.nan,
                'TRUK/BUS': np.nan,
                'JUMLAH': np.nan,
                'GAP (s)': np.nan,
                'HEADWAY (s)': np.nan,
                'AVG. SPEED (Km/h)': np.nan,
                'OCCUPANCY (%)': np.nan,
                'SHEET NAME': str(day.day).zfill(2)
            })
            missing_times_list.append(missing_times_df)
    
    # Menggabungkan semua DataFrame dari missing_times_list menjadi satu DataFrame
    missing_times = pd.concat(missing_times_list, ignore_index=True) if missing_times_list else pd.DataFrame()
    missing_times = pisahkan_kolom_waktu(missing_times)
    missing_times['JUMLAH'] = np.nan
    
    # # Menampilkan data yang hilang
    if missing_times.empty:
        print("Tidak ada data yang hilang.")
    else:
        print("Ada data yang hilang")
        # print(missing_times)
    
    missing_times.to_excel('../Data/Preprocessing/Data Balonggandu/2023/1_missing_times_'+nama_file, index=False)

    return df

def menggisi_data_hilang(df, missing_times):
    # Menggabungkan data hilang dengan DataFrame utama
    df = pd.concat([df, missing_times], ignore_index=True)
    
    # Mengurutkan DataFrame berdasarkan kolom 'Waktu'
    df = df.sort_values(by='WAKTU').reset_index(drop=True)

    return df

def perhitungan_smp(df):
    df['SMP_MTR'] = df['MOTOR'] * 0.2
    df['SMP_MBL'] = df['MOBIL']
    df['SMP_TRK'] = df['TRUK/BUS'] * 2.5
    df['SMP'] = round(df['SMP_MTR'] + df['SMP_MBL'] + df['SMP_TRK'])

    return df

In [10]:
# Contoh penggunaan
excel_files = ['../Data/DataLHR/Data Balonggandu/2023/09 September.xlsx']

all_dfs = []
for file in excel_files:
    bulan = '09'
    tahun = '2024'
    thn_bln = f'{tahun}-{bulan}-'
    
    result = read_excel_until_total(file)
    result = penyesuaian_format(result)
    result = pisahkan_kolom_waktu(result)
    result = perhitungan_smp(result)
    
       
    # Tambahkan DataFrame ke list
    all_dfs.append(result)

df_combined = pd.concat(all_dfs)
df_combined = df_combined.sort_values(by='WAKTU')

df_combined.head()


,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2024-09-01 00:00:00,2024,9,1,Sunday,00:00,13,9,6,28,2.6,9,15.0,27.0
1,2024-09-01 01:00:00,2024,9,1,Sunday,01:00,7,5,7,19,1.4,5,17.5,24.0
2,2024-09-01 02:00:00,2024,9,1,Sunday,02:00,6,15,11,32,1.2,15,27.5,44.0
3,2024-09-01 03:00:00,2024,9,1,Sunday,03:00,3,8,9,20,0.6,8,22.5,31.0
4,2024-09-01 04:00:00,2024,9,1,Sunday,04:00,15,19,8,42,3.0,19,20.0,42.0


In [11]:
nilai_awal = df_combined['WAKTU'].iloc[0]   # Nilai awal (baris pertama)
nilai_akhir = nilai_awal + pd.offsets.MonthEnd(0)

In [12]:
print(nilai_awal, nilai_akhir)

2024-09-01 00:00:00 2024-09-30 00:00:00


In [13]:
df = mencari_data_hilang(nilai_awal,nilai_akhir,df_combined,'09 september.xlsx')

Ada data yang hilang


In [14]:
df_combined.to_excel('../Data/Preprocessing/Data Balonggandu/2023/1_input_09 september.xlsx', index=False)